# S&OP Pre-Build Production Optimization

## Business Objective

Manufacturing organizations face a critical challenge: **how to build inventory ahead of peak demand seasons (Q4) without exceeding warehouse capacity or incurring excessive storage costs**. This notebook implements a Linear Programming optimization model to calculate the optimal Q3 production schedule that:

1. **Meets Q4 demand** without stockouts
2. **Avoids warehouse overflow** and penalty fees
3. **Smoothes production** to avoid utilization spikes
4. **Minimizes total cost of carry**

## Technical Approach

We use **Linear Programming (LP)** via the PuLP library to solve a multi-period production planning problem.

## Learning Objectives

After completing this notebook, you will understand:
1. How to formulate production planning as a Linear Programming problem
2. How to handle multi-period inventory balance constraints
3. How to interpret optimization results for business decision-making

## Prerequisites

- **Mathematics**: Basic linear algebra, understanding of optimization constraints
- **Python**: Pandas, NumPy fundamentals
- **Domain**: S&OP planning concepts, inventory management basics

## Output

This notebook produces:
- `SOP_LOGISTICS.RECOMMENDED_BUILD_PLAN` table with optimized production schedule


## 1. Environment Setup

Install required packages and configure the Snowflake session.


In [ ]:
# Install optimization library
import os
import sys

packages = ["pulp"]
for pkg in packages:
    os.system(f"{sys.executable} -m pip install {pkg} -q")

print("✅ Packages installed")


In [ ]:
# =============================================================================
# IMPORTS
# =============================================================================
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Optimization
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value

# Visualization
import matplotlib.pyplot as plt

# Snowflake
from snowflake.snowpark.context import get_active_session

print("✅ Libraries imported")


In [ ]:
# =============================================================================
# VISUALIZATION CONFIGURATION - Dark theme with colorblind-safe palette
# =============================================================================
plt.style.use('dark_background')
plt.rcParams.update({
    'figure.facecolor': '#121212',
    'axes.facecolor': '#121212',
    'text.color': '#E5E5E7',
    'axes.labelcolor': '#E5E5E7',
    'xtick.color': '#A1A1A6',
    'ytick.color': '#A1A1A6',
    'axes.edgecolor': '#3A3A3C',
    'grid.color': '#2C2C2E',
    'grid.alpha': 0.6,
    'lines.linewidth': 2,
    'figure.dpi': 150,
    'figure.figsize': (12, 6),
})

COLORS = {
    'primary': '#64D2FF',
    'secondary': '#FF9F0A',
    'accent1': '#5AC8FA',
    'danger': '#FF6B6B',
    'baseline': '#A1A1A6'
}

print("✅ Visualization configured")


In [ ]:
# =============================================================================
# SNOWFLAKE SESSION
# =============================================================================
session = get_active_session()

result = session.sql("SELECT CURRENT_DATABASE(), CURRENT_SCHEMA(), CURRENT_WAREHOUSE()").collect()
print(f"📦 Database: {result[0][0]}")
print(f"📋 Schema: {result[0][1]}")
print(f"⚡ Warehouse: {result[0][2]}")


In [ ]:
# =============================================================================
# HELPER FUNCTIONS
# =============================================================================
def execute_query(session, query: str, name: str = "query") -> pd.DataFrame:
    """Execute SQL query with fail-fast error handling."""
    try:
        result = session.sql(query).to_pandas()
        if result is None:
            raise RuntimeError(f"Query '{name}' returned None")
        return result
    except Exception as e:
        raise RuntimeError(f"Query '{name}' failed: {e}") from e

def safe_float(value: float, default: float = 0.0) -> float:
    """Convert model output to safe float for database storage."""
    import math
    if value is None or math.isnan(value) or math.isinf(value):
        return default
    return float(value)

print("✅ Helper functions defined")


## 2. Data Loading

Load demand forecasts, production capacity, and warehouse constraints from Snowflake.


In [ ]:
# =============================================================================
# LOAD DEMAND FORECASTS
# =============================================================================
demand_query = """
SELECT 
    SCENARIO_CODE,
    FISCAL_MONTH,
    FISCAL_QUARTER,
    SUM(FORECAST_QUANTITY) as TOTAL_DEMAND,
    SUM(TOTAL_REVENUE) as TOTAL_REVENUE
FROM SOP_LOGISTICS.SCENARIO_COMPARISON_V
WHERE SCENARIO_CODE IN ('BASELINE', 'Q4_PUSH')
GROUP BY SCENARIO_CODE, FISCAL_MONTH, FISCAL_QUARTER
ORDER BY SCENARIO_CODE, FISCAL_MONTH
"""

demand_df = execute_query(session, demand_query, "load_demand_forecasts")

print(f"📊 Loaded {len(demand_df)} demand forecast rows")
print(f"   Scenarios: {demand_df['SCENARIO_CODE'].unique().tolist()}")
demand_df.head(10)


In [ ]:
# =============================================================================
# LOAD PRODUCTION AND WAREHOUSE CAPACITY
# =============================================================================
capacity_query = """
SELECT 
    SUM(MAX_DAILY_CAPACITY) as DAILY_CAPACITY,
    SUM(MAX_MONTHLY_CAPACITY) as MONTHLY_CAPACITY
FROM SOP_LOGISTICS.PRODUCTION_CAPACITY_SUMMARY
"""
capacity_df = execute_query(session, capacity_query, "load_production_capacity")
MONTHLY_CAPACITY = int(capacity_df['MONTHLY_CAPACITY'].iloc[0])

warehouse_query = """
SELECT 
    SUM(wz.MAX_CAPACITY_PALLETS) as TOTAL_CAPACITY_PALLETS,
    SUM(wz.CURRENT_OCCUPANCY_PALLETS) as CURRENT_OCCUPANCY_PALLETS
FROM ATOMIC.WAREHOUSE_ZONE wz
JOIN ATOMIC.SITE s ON wz.SITE_ID = s.SITE_ID
WHERE s.SITE_TYPE = 'WAREHOUSE' AND wz.IS_CURRENT_FLAG = TRUE AND s.IS_CURRENT_FLAG = TRUE
"""
warehouse_df = execute_query(session, warehouse_query, "load_warehouse_capacity")

UNITS_PER_PALLET = 10
WAREHOUSE_CAPACITY_UNITS = int(warehouse_df['TOTAL_CAPACITY_PALLETS'].iloc[0]) * UNITS_PER_PALLET
CURRENT_INVENTORY_UNITS = int(warehouse_df['CURRENT_OCCUPANCY_PALLETS'].iloc[0]) * UNITS_PER_PALLET
OVERFLOW_THRESHOLD_PCT = 0.87
MAX_SAFE_INVENTORY = int(WAREHOUSE_CAPACITY_UNITS * OVERFLOW_THRESHOLD_PCT)

print(f"🏭 Production Capacity: {MONTHLY_CAPACITY:,} units/month")
print(f"🏬 Warehouse Capacity: {WAREHOUSE_CAPACITY_UNITS:,} units")
print(f"   Current Inventory: {CURRENT_INVENTORY_UNITS:,} units")
print(f"   Overflow Threshold: {MAX_SAFE_INVENTORY:,} units")


## 3. Data Exploration

Visualize demand patterns and the "camel hump" inventory buildup curve.


In [ ]:
# =============================================================================
# DEMAND COMPARISON VISUALIZATION
# =============================================================================
demand_pivot = demand_df.pivot(index='FISCAL_MONTH', columns='SCENARIO_CODE', values='TOTAL_DEMAND')
month_order = ['July', 'August', 'September', 'October', 'November', 'December']
demand_pivot = demand_pivot.reindex(month_order)

fig, ax = plt.subplots(figsize=(12, 5))
x = range(len(month_order))
width = 0.35

ax.bar([i - width/2 for i in x], demand_pivot['BASELINE'], width, label='Baseline', color=COLORS['baseline'], alpha=0.8)
ax.bar([i + width/2 for i in x], demand_pivot['Q4_PUSH'], width, label='Q4 Push', color=COLORS['secondary'], alpha=0.8)
ax.axvline(x=2.5, color=COLORS['danger'], linestyle='--', alpha=0.5, label='Q3/Q4 Boundary')
ax.set_xlabel('Month')
ax.set_ylabel('Demand (Units)')
ax.set_title('Monthly Demand: Baseline vs Q4 Marketing Push')
ax.set_xticks(x)
ax.set_xticklabels(month_order, rotation=45)
ax.legend()
plt.tight_layout()
plt.show()

pct_increase = ((demand_pivot['Q4_PUSH'] - demand_pivot['BASELINE']) / demand_pivot['BASELINE'] * 100)
print(f"\n📊 Q4 Average Demand Increase: {pct_increase[3:].mean():.1f}%")


## 4. Optimization Model

### Linear Programming Formulation

**Objective:** Minimize total cost = production cost + storage cost

**Decision Variables:**
- P[t] = Production quantity in period t
- I[t] = Inventory at end of period t

**Constraints:**
1. Inventory Balance: I[t] = I[t-1] + P[t] - D[t]
2. Production Capacity: P[t] ≤ C_max
3. Warehouse Capacity: I[t] ≤ W_max
4. Safety Stock: I[t] ≥ SAFETY_STOCK_RATIO × D[t+1] (for t < NUM_PERIODS-1)


In [ ]:
# =============================================================================
# PREPARE OPTIMIZATION DATA - Focus on Q4_PUSH scenario
# =============================================================================
q4_push_demand = demand_pivot['Q4_PUSH'].to_dict()
PERIODS = month_order
NUM_PERIODS = len(PERIODS)
DEMAND = [int(q4_push_demand[m]) for m in PERIODS]
PRODUCTION_COST = 1.0
STORAGE_COST = 0.22  # Per unit per month
INITIAL_INVENTORY = CURRENT_INVENTORY_UNITS
SAFETY_STOCK_RATIO = 0.15  # 15% of next period's demand as safety stock

print("📋 Optimization Parameters:")
print(f"   Periods: {PERIODS}")
print(f"   Demand: {DEMAND}")
print(f"   Monthly Capacity: {MONTHLY_CAPACITY:,}")
print(f"   Max Inventory: {MAX_SAFE_INVENTORY:,}")
print(f"   Storage Cost: ${STORAGE_COST}/unit/month")
print(f"   Safety Stock Ratio: {SAFETY_STOCK_RATIO:.0%}")


In [ ]:
# =============================================================================
# DEFINE LINEAR PROGRAMMING MODEL
# =============================================================================
model = LpProblem("PreBuild_Production_Optimization", LpMinimize)

# Decision Variables
Production = [LpVariable(f"Production_{t}", lowBound=0, upBound=MONTHLY_CAPACITY) for t in range(NUM_PERIODS)]
Inventory = [LpVariable(f"Inventory_{t}", lowBound=0, upBound=MAX_SAFE_INVENTORY) for t in range(NUM_PERIODS)]

# Objective Function: Minimize production + storage cost
model += lpSum([PRODUCTION_COST * Production[t] + STORAGE_COST * Inventory[t] for t in range(NUM_PERIODS)]), "Total_Cost"

# Constraints: Inventory Balance
for t in range(NUM_PERIODS):
    if t == 0:
        model += Inventory[t] == INITIAL_INVENTORY + Production[t] - DEMAND[t], f"Inv_Balance_{t}"
    else:
        model += Inventory[t] == Inventory[t-1] + Production[t] - DEMAND[t], f"Inv_Balance_{t}"
    model += Inventory[t] >= 0, f"Min_Inv_{t}"

# Constraints: Safety Stock - Maintain buffer for demand variability
# This ensures we pre-build inventory in Q3 to meet Q4 demand surge
for t in range(NUM_PERIODS - 1):  # Not needed for last period (December)
    safety_stock = int(DEMAND[t + 1] * SAFETY_STOCK_RATIO)
    model += Inventory[t] >= safety_stock, f"Safety_Stock_{t}"

print(f"✅ LP Model defined with {len(Production) + len(Inventory)} variables and {len(model.constraints)} constraints")


## 5. Solve & Evaluate

Execute the optimization and analyze the recommended production schedule.


In [ ]:
# =============================================================================
# SOLVE THE OPTIMIZATION MODEL
# =============================================================================
status = model.solve()
print(f"🔧 Optimization Status: {LpStatus[status]}")
print(f"💰 Optimal Total Cost: ${value(model.objective):,.2f}")

if LpStatus[status] != 'Optimal':
    raise RuntimeError(f"Optimization failed with status: {LpStatus[status]}")


In [ ]:
# =============================================================================
# EXTRACT AND DISPLAY RESULTS
# =============================================================================
results = []
for t in range(NUM_PERIODS):
    prod_val = value(Production[t])
    inv_val = value(Inventory[t])
    util_pct = (prod_val / MONTHLY_CAPACITY) * 100 if MONTHLY_CAPACITY > 0 else 0
    wh_util = (inv_val / MAX_SAFE_INVENTORY) * 100 if MAX_SAFE_INVENTORY > 0 else 0
    
    results.append({
        'PERIOD': t + 1,
        'FISCAL_MONTH': PERIODS[t],
        'FISCAL_QUARTER': 'Q3' if t < 3 else 'Q4',
        'DEMAND': DEMAND[t],
        'RECOMMENDED_PRODUCTION': int(prod_val),
        'ENDING_INVENTORY': int(inv_val),
        'CAPACITY_UTILIZATION_PCT': round(util_pct, 1),
        'WAREHOUSE_UTILIZATION_PCT': round(wh_util, 1),
        'PRODUCTION_COST': round(prod_val * PRODUCTION_COST, 2),
        'STORAGE_COST': round(inv_val * STORAGE_COST, 2)
    })

results_df = pd.DataFrame(results)
print("📋 Optimized Production Schedule:")
results_df


In [ ]:
# =============================================================================
# VISUALIZE OPTIMIZATION RESULTS
# =============================================================================
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
x = range(NUM_PERIODS)

# Plot 1: Production vs Demand
width = 0.35
axes[0, 0].bar([i - width/2 for i in x], results_df['DEMAND'], width, label='Demand', color=COLORS['secondary'], alpha=0.8)
axes[0, 0].bar([i + width/2 for i in x], results_df['RECOMMENDED_PRODUCTION'], width, label='Production', color=COLORS['primary'], alpha=0.8)
axes[0, 0].axhline(y=MONTHLY_CAPACITY, color=COLORS['danger'], linestyle='--', alpha=0.7, label='Max Capacity')
axes[0, 0].set_title('Recommended Production vs Demand')
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(PERIODS, rotation=45)
axes[0, 0].legend()

# Plot 2: Inventory Level
axes[0, 1].fill_between(x, results_df['ENDING_INVENTORY'], alpha=0.3, color=COLORS['primary'])
axes[0, 1].plot(x, results_df['ENDING_INVENTORY'], marker='o', color=COLORS['primary'], linewidth=2)
axes[0, 1].axhline(y=MAX_SAFE_INVENTORY, color=COLORS['danger'], linestyle='--', alpha=0.7, label='Overflow Threshold')
axes[0, 1].set_title('Optimized Inventory Level')
axes[0, 1].set_xticks(x)
axes[0, 1].set_xticklabels(PERIODS, rotation=45)
axes[0, 1].legend()

# Plot 3: Capacity Utilization
colors = [COLORS['primary'] if u < 90 else COLORS['secondary'] for u in results_df['CAPACITY_UTILIZATION_PCT']]
axes[1, 0].bar(x, results_df['CAPACITY_UTILIZATION_PCT'], color=colors, alpha=0.8)
axes[1, 0].axhline(y=100, color=COLORS['danger'], linestyle='--', alpha=0.7)
axes[1, 0].set_title('Production Capacity Utilization')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(PERIODS, rotation=45)
axes[1, 0].set_ylim(0, 110)

# Plot 4: Cumulative Cost
results_df['CUMULATIVE_COST'] = (results_df['PRODUCTION_COST'] + results_df['STORAGE_COST']).cumsum()
axes[1, 1].plot(x, results_df['CUMULATIVE_COST'], marker='s', color=COLORS['accent1'], linewidth=2)
axes[1, 1].fill_between(x, results_df['CUMULATIVE_COST'], alpha=0.3, color=COLORS['accent1'])
axes[1, 1].set_title('Cumulative Cost')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(PERIODS, rotation=45)

plt.tight_layout()
plt.show()

print(f"\n📊 Summary: Peak Inventory = {results_df['ENDING_INVENTORY'].max():,} units, Avg Utilization = {results_df['CAPACITY_UTILIZATION_PCT'].mean():.1f}%")


## 6. Production Output

Write the optimized production schedule to Snowflake.


In [ ]:
# =============================================================================
# PREPARE OUTPUT FOR SNOWFLAKE
# =============================================================================
scenario_df = execute_query(session, "SELECT SCENARIO_ID FROM ATOMIC.SCENARIO_DEFINITION WHERE SCENARIO_CODE = 'Q4_PUSH'", "get_scenario")
Q4_PUSH_SCENARIO_ID = int(scenario_df['SCENARIO_ID'].iloc[0])

wc_df = execute_query(session, "SELECT MIN(WORK_CENTER_ID) as WC_ID FROM ATOMIC.WORK_CENTER WHERE IS_CURRENT_FLAG = TRUE", "get_wc")
WORK_CENTER_ID = int(wc_df['WC_ID'].iloc[0])

prod_df = execute_query(session, "SELECT MIN(PRODUCT_ID) as PROD_ID FROM ATOMIC.PRODUCT WHERE IS_CURRENT_FLAG = TRUE", "get_prod")
PRODUCT_ID = int(prod_df['PROD_ID'].iloc[0])

model_version = f"LP_v1.0_{datetime.now().strftime('%Y%m%d')}"
output_records = []

for _, row in results_df.iterrows():
    month_num = PERIODS.index(row['FISCAL_MONTH']) + 7
    production_week = datetime(2024, month_num, 1)
    
    output_records.append({
        'PRODUCT_ID': PRODUCT_ID, 'WORK_CENTER_ID': WORK_CENTER_ID, 'SCENARIO_ID': Q4_PUSH_SCENARIO_ID,
        'PRODUCTION_WEEK': production_week.strftime('%Y-%m-%d'), 'FISCAL_MONTH': row['FISCAL_MONTH'],
        'FISCAL_QUARTER': row['FISCAL_QUARTER'], 'RECOMMENDED_QUANTITY': int(row['RECOMMENDED_PRODUCTION']),
        'CURRENT_CAPACITY_AVAILABLE': int(MONTHLY_CAPACITY), 'CAPACITY_UTILIZATION_PCT': safe_float(row['CAPACITY_UTILIZATION_PCT']),
        'PROJECTED_INVENTORY': int(row['ENDING_INVENTORY']), 'WAREHOUSE_UTILIZATION_PCT': safe_float(row['WAREHOUSE_UTILIZATION_PCT']),
        'PROJECTED_PRODUCTION_COST': safe_float(row['PRODUCTION_COST']), 'PROJECTED_STORAGE_COST': safe_float(row['STORAGE_COST']),
        'PROJECTED_TOTAL_COST': safe_float(row['PRODUCTION_COST'] + row['STORAGE_COST']),
        'MODEL_VERSION': model_version, 'MODEL_CONFIDENCE': 0.95
    })

output_df = pd.DataFrame(output_records)
print(f"📦 Prepared {len(output_df)} records for output")


In [ ]:
# =============================================================================
# WRITE RESULTS TO SNOWFLAKE
# =============================================================================
session.sql(f"DELETE FROM SOP_LOGISTICS.RECOMMENDED_BUILD_PLAN WHERE SCENARIO_ID = {Q4_PUSH_SCENARIO_ID}").collect()

# Use explicit column names to avoid issues with IDENTITY and DEFAULT columns
columns = [
    'PRODUCT_ID', 'WORK_CENTER_ID', 'SCENARIO_ID', 'PRODUCTION_WEEK',
    'FISCAL_MONTH', 'FISCAL_QUARTER', 'RECOMMENDED_QUANTITY',
    'CURRENT_CAPACITY_AVAILABLE', 'CAPACITY_UTILIZATION_PCT',
    'PROJECTED_INVENTORY', 'WAREHOUSE_UTILIZATION_PCT',
    'PROJECTED_PRODUCTION_COST', 'PROJECTED_STORAGE_COST',
    'PROJECTED_TOTAL_COST', 'MODEL_VERSION', 'MODEL_CONFIDENCE'
]

# Insert rows using SQL with explicit column list
for _, row in output_df.iterrows():
    insert_sql = f"""
        INSERT INTO SOP_LOGISTICS.RECOMMENDED_BUILD_PLAN (
            {', '.join(columns)}
        ) VALUES (
            {row['PRODUCT_ID']},
            {row['WORK_CENTER_ID']},
            {row['SCENARIO_ID']},
            '{row['PRODUCTION_WEEK']}',
            '{row['FISCAL_MONTH']}',
            '{row['FISCAL_QUARTER']}',
            {row['RECOMMENDED_QUANTITY']},
            {row['CURRENT_CAPACITY_AVAILABLE']},
            {row['CAPACITY_UTILIZATION_PCT']},
            {row['PROJECTED_INVENTORY']},
            {row['WAREHOUSE_UTILIZATION_PCT']},
            {row['PROJECTED_PRODUCTION_COST']},
            {row['PROJECTED_STORAGE_COST']},
            {row['PROJECTED_TOTAL_COST']},
            '{row['MODEL_VERSION']}',
            {row['MODEL_CONFIDENCE']}
        )
    """
    session.sql(insert_sql).collect()

verify_df = execute_query(session, "SELECT COUNT(*) as CNT FROM SOP_LOGISTICS.RECOMMENDED_BUILD_PLAN", "verify_output")
row_count = int(verify_df['CNT'].iloc[0])

if row_count < len(output_df):
    raise RuntimeError(f"Output verification failed: wrote {len(output_df)} rows, but table contains {row_count}")

print(f"✅ Successfully wrote {len(output_df)} records to RECOMMENDED_BUILD_PLAN")


## 7. Key Takeaways

### What the Model Learned
1. **Production Smoothing**: The optimizer spreads production across Q3 to avoid utilization spikes in Q4
2. **Pre-Build Strategy**: Building inventory in August-September reduces Q4 peak demand on production
3. **Warehouse Utilization**: The model keeps inventory below the 87% threshold to avoid overflow penalties

### Interpretation Guidelines
| Output Column | Value Range | Interpretation |
|--------------|-------------|----------------|
| RECOMMENDED_QUANTITY | 0 - MONTHLY_CAPACITY | Optimal production units |
| CAPACITY_UTILIZATION_PCT | 0% - 100% | >85% = high utilization |
| WAREHOUSE_UTILIZATION_PCT | 0% - 87% | Values near 87% = capacity risk |

### Limitations
- Simplified model: aggregates all products
- Single scenario optimization (Q4_PUSH)
- Static capacity assumption


In [ ]:
# =============================================================================
# FINAL SUMMARY
# =============================================================================
print("="*60)
print("📊 OPTIMIZATION COMPLETE")
print("="*60)
print(f"\n✅ Model: Linear Programming (PuLP)")
print(f"✅ Status: {LpStatus[status]}")
print(f"✅ Total Cost: ${value(model.objective):,.2f}")
print(f"\n📋 Output: SOP_LOGISTICS.RECOMMENDED_BUILD_PLAN")
print(f"   Records Written: {len(output_df)}")
print(f"   Model Version: {model_version}")
print(f"\n🎯 Key Insights:")
print(f"   - Peak production: {results_df.loc[results_df['RECOMMENDED_PRODUCTION'].idxmax(), 'FISCAL_MONTH']}")
print(f"   - Peak inventory: {results_df.loc[results_df['ENDING_INVENTORY'].idxmax(), 'FISCAL_MONTH']}")
print(f"   - Overflow avoided: {'Yes' if results_df['WAREHOUSE_UTILIZATION_PCT'].max() < 100 else 'No'}")
print("="*60)
